OVERVIEW - this Notebook will be used to complete the IBM Data Science Certificate Capstone Project. Project will include 3 sections as follows (i) Scraping Data from a Wikipedia Page (ii) Establishing a dataframe which includes location data for neigborhoods such as latitudes and longitudes  (iii) performing clustering/segmentation of the neighborhood and creating map visualizations

In [8]:
# importing initial python libraries
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


# PART I - SCRAPING DATA FROM WIKIPEDIA PAGE

In [5]:
#installing necessary packages and the Requests Library. Using BeautifulSoup to scrape the wikipage
!pip install bs4
import requests
from bs4 import BeautifulSoup

The wiki page 'https://wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' contains list of postal codes. Postal codes beginning with M are located within the city of Toronto in the province of Ontario.

In [9]:
#downloading source code for wiki page and creating the soup object
url= 'https://wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url)
soup = BeautifulSoup(html_data.content)
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

Parsing through the web page code and extracting the relevant information which is contained in the class attributes ('td'), skipping rows that do not have assigned postalcodes

In [10]:

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)

df.shape

(103, 3)

In [11]:
#cleaning up some data for boroughs that have unusual names
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [12]:
df.tail(20)

,PostalCode,Borough,Neighborhood
83,M4T,Central Toronto,"Moore Park, Summerhill East"
84,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
85,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo..."
86,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
87,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores"
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
90,M1W,Scarborough,"Steeles West, L'Amoreaux West"
91,M4W,Downtown Toronto,Rosedale
92,M5W,Downtown Toronto Stn A,Enclave of M5E


In [13]:
df.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [14]:
#size of dataframe
df.shape

(103, 3)

# PART II - IMPORTING GEOSPATIAL INFORMATION FOR ALL NEIGHBORHOODS#

In [15]:
#import required packages
!pip install geopy
from geopy.geocoders import Nominatim


In [16]:
geolocator=Nominatim(user_agent="foursquare_agent")


In [11]:

import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

ModuleNotFoundError: No module named 'geocoder'

In [17]:
#Downloading data from a link to a file containing coordinates for all postal codes in Toronto
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
locationData = "Geospatial_Coordinates.csv"

--2021-06-03 14:44:04--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2788 (2.7K) [text/csv]
Saving to: ‘Geospatial_Coordinates.csv.1’

Geospatial_Coordina 100%[===================>]   2.72K  --.-KB/s    in 0s      

2021-06-03 14:44:04 (88.5 MB/s) - ‘Geospatial_Coordinates.csv.1’ saved [2788/2788]



In [18]:
#open, read and print contents of the CSV file
with open ("Geospatial_Coordinates.csv",'r') as file1:
    print(file1.read())

Postal Code,Latitude,Longitude
M1B,43.8066863,-79.1943534
M1C,43.7845351,-79.1604971
M1E,43.7635726,-79.1887115
M1G,43.7709921,-79.2169174
M1H,43.773136,-79.2394761
M1J,43.7447342,-79.2394761
M1K,43.7279292,-79.2620294
M1L,43.7111117,-79.2845772
M1M,43.716316,-79.2394761
M1N,43.692657,-79.2648481
M1P,43.7574096,-79.273304
M1R,43.7500715,-79.2958491
M1S,43.7942003,-79.2620294
M1T,43.7816375,-79.3043021
M1V,43.8152522,-79.2845772
M1W,43.7995252,-79.3183887
M1X,43.8361247,-79.2056361
M2H,43.8037622,-79.3634517
M2J,43.7785175,-79.3465557
M2K,43.7869473,-79.385975
M2L,43.7574902,-79.3747141
M2M,43.789053,-79.4084928
M2N,43.7701199,-79.4084928
M2P,43.7527583,-79.4000493
M2R,43.7827364,-79.4422593
M3A,43.7532586,-79.3296565
M3B,43.7459058,-79.352188
M3C,43.7258997,-79.340923
M3H,43.7543283,-79.4422593
M3J,43.7679803,-79.4872619
M3K,43.7374732,-79.4647633
M3L,43.7390146,-79.5069436
M3M,43.7284964,-79.4956974
M3N,43.7616313,-79.5209994
M4A,43.7258823,-79.3155716
M4B,43.7063972,-79.309937
M4C,43

In [19]:
#Convert CSV file into Panda Dataframe
df2=pd.read_csv(locationData)
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
#sort the original dataframe (df) by PostalCode so it is consistent with Df2
df_sorted=df.sort_values("PostalCode").reset_index()
df_sorted.head()
df_sorted.shape

(103, 4)

In [21]:
#merge the two dataframes and drop the original index column
df_merged=df_sorted
df_merged[['Latitude','Longitude']]=df2[['Latitude','Longitude']]
df_merged.drop('index', axis=1, inplace=True)
df_merged.tail()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
102,M9W,Etobicoke Northwest,"Clairville, Humberwood, Woodbine Downs, West H...",43.706748,-79.594054


In [22]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_merged['Borough'].unique()),
        df_merged.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


# Replicating the Analysis performed for NYC using information scrapped for Toronto Neighborhoods

In [15]:
!pip install -U numpy

!pip install -U pandas

!pip install -U scipy==1.4.1

!pip install -U scikit-learn

!pip install -U imbalanced-learn
!pip install sklearn

     |████████████████████████████████| 26.1 MB 19.3 MB/s eta 0:00:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.3
    Uninstalling scipy-1.5.3:
      Successfully uninstalled scipy-1.5.3
     |████████████████████████████████| 206 kB 19.3 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=b26b7f971efab1e0a813daf2c7ab22d3c23c273bca7d0133a7b9ebfd6d2f4703
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1


In [2]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="cnd_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [23]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [24]:
#Define Foursquare Credentials and Version
CLIENT_ID = '22YQLBTXULNITLRYJTROKEG55KFSQPD2UUQUNBWOFA24Z3W5' # your Foursquare ID
CLIENT_SECRET = '3QHPDYPEIF2MWJRTXX3TYAM102S1QC5FRA1GNNTHISUNO4JN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 22YQLBTXULNITLRYJTROKEG55KFSQPD2UUQUNBWOFA24Z3W5
CLIENT_SECRET:3QHPDYPEIF2MWJRTXX3TYAM102S1QC5FRA1GNNTHISUNO4JN


Let's explore the first neighborhood in our dataframe.

In [27]:
df_merged.loc[0, 'Neighborhood']


'Malvern, Rouge'

In [28]:
#Get the neighborhood's latitude and longitude values.
neighborhood_latitude = df_merged.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_merged.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_merged.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.8066863, -79.1943534.


Now, let's get the top 100 venues that are in Malvern, Rouge within a radius of 1000 meters.

In [31]:
# First, let's create the GET request URL. Name your URL url.

CLIENT_ID = '22YQLBTXULNITLRYJTROKEG55KFSQPD2UUQUNBWOFA24Z3W5' # your Foursquare ID
CLIENT_SECRET = '3QHPDYPEIF2MWJRTXX3TYAM102S1QC5FRA1GNNTHISUNO4JN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=22YQLBTXULNITLRYJTROKEG55KFSQPD2UUQUNBWOFA24Z3W5&client_secret=3QHPDYPEIF2MWJRTXX3TYAM102S1QC5FRA1GNNTHISUNO4JN&v=20180605&ll=43.8066863,-79.1943534&radius=1000&limit=100'

Send the GET request and examine the resutls

In [32]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60b8f01a01e5b1543302cc7b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 21,
  'suggestedBounds': {'ne': {'lat': 43.815686309000014,
    'lng': -79.1819057614608},
   'sw': {'lat': 43.79768629099999, 'lng': -79.2068010385392}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs': 

All the information is in the items key

In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe.

In [34]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Harvey's,Restaurant,43.800020,-79.198307
2,RBC Royal Bank,Bank,43.798782,-79.197090
3,Wendy's,Fast Food Restaurant,43.802008,-79.198080
4,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [35]:
# how many venues were returned by Foursquare?
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

21 venues were returned by Foursquare.


In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues

In [44]:
toronto_venues = getNearbyVenues(names=df_merged['Neighborhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West, Riverdale


In [45]:
#how many venues by each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,48,48,48,48,48,48
"Alderwood, Long Branch",27,27,27,27,27,27
"Bathurst Manor, Wilson Heights, Downsview North",29,29,29,29,29,29
Bayview Village,18,18,18,18,18,18
"Bedford Park, Lawrence Manor East",44,44,44,44,44,44
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",13,13,13,13,13,13
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15


In [46]:
#Let's find out how many unique categories can be curated from all the returned venues¶
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 336 uniques categories.


Analyze each Neighborhod

In [47]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink

In [48]:
toronto_onehot.shape

(4903, 336)

let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [49]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink

For each neighborhood the 5 most common venues

In [50]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.15
1         Shopping Mall  0.08
2                Bakery  0.04
3        Sandwich Place  0.04
4  Caribbean Restaurant  0.04


----Alderwood, Long Branch----
            venue  freq
0  Discount Store  0.11
1     Pizza Place  0.07
2            Park  0.07
3        Pharmacy  0.07
4      Print Shop  0.04


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0     Coffee Shop  0.07
1            Bank  0.07
2  Ice Cream Shop  0.03
3        Ski Area  0.03
4     Men's Store  0.03


----Bayview Village----
                 venue  freq
0        Grocery Store  0.11
1  Japanese Restaurant  0.11
2                 Bank  0.11
3          Gas Station  0.11
4         Intersection  0.11


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.07
2         Pizza Place  0.05
3                Bank  0.05
4          Restaurant  0.05


----Bercz

In [103]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create the new dataframe and display the top 10 venues for each neighborhood.

In [102]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Bakery,Sandwich Place,Caribbean Restaurant,Indian Restaurant,Cantonese Restaurant,Mediterranean Restaurant,Sushi Restaurant,Supermarket
1,"Alderwood, Long Branch",Discount Store,Pizza Place,Park,Pharmacy,Print Shop,Skating Rink,Shopping Mall,Liquor Store,Sandwich Place,Grocery Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Ice Cream Shop,Ski Area,Men's Store,Sushi Restaurant,Middle Eastern Restaurant,Gas Station,Mobile Phone Shop,Bridal Shop
3,Bayview Village,Grocery Store,Japanese Restaurant,Bank,Gas Station,Intersection,Shopping Mall,Park,Skating Rink,Trail,Café
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Pizza Place,Bank,Restaurant,Fast Food Restaurant,Sandwich Place,Greek Restaurant,Pet Store,Thai Restaurant


Run k-means to cluster the neighborhood into 5 clusters.

In [104]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 1, 1, 1, 1, 1, 3], dtype=int32)

In [105]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_merged

# merge toronto_grouped with df_merged to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3.0,Fast Food Restaurant,Coffee Shop,Trail,Bank,Hobby Shop,Gym,Chinese Restaurant,Sandwich Place,Paper / Office Supplies Store,Park
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,Italian Restaurant,Breakfast Spot,Park,Burger Joint,Playground,Newsagent,Organic Grocery,Optical Shop,Opera House,Office
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3.0,Pizza Place,Fast Food Restaurant,Bank,Fried Chicken Joint,Park,Food & Drink Shop,Sports Bar,Smoothie Shop,Supermarket,Beer Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Park,Coffee Shop,Chinese Restaurant,Indian Restaurant,Pharmacy,Fast Food Restaurant,Mobile Phone Shop,Music Store,Other Great Outdoors,Museum
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3.0,Bakery,Indian Restaurant,Gas Station,Coffee Shop,Bank,Grocery Store,Fried Chicken Joint,Music Store,Fast Food Restaurant,Pharmacy


In [107]:
toronto_merged['Cluster Labels'][0:50]

0     3.0
1     1.0
2     3.0
3     0.0
4     3.0
5     3.0
6     3.0
7     3.0
8     3.0
9     1.0
10    3.0
11    3.0
12    3.0
13    3.0
14    3.0
15    3.0
16    NaN
17    3.0
18    1.0
19    3.0
20    0.0
21    1.0
22    1.0
23    3.0
24    3.0
25    3.0
26    1.0
27    1.0
28    3.0
29    1.0
30    1.0
31    0.0
32    4.0
33    3.0
34    1.0
35    3.0
36    1.0
37    1.0
38    1.0
39    3.0
40    1.0
41    1.0
42    1.0
43    1.0
44    1.0
45    1.0
46    1.0
47    1.0
48    1.0
49    1.0
Name: Cluster Labels, dtype: float64

In [108]:
#toronto_merged['Cluster Labels']= toronto_merged['Cluster Labels'].apply(pd.to_numeric) 
#toronto_merged['Cluster Labels'].value_counts()
toronto_merged['Cluster Labels'][16]=1  #changing the cell value from Nan to 1, which is the most frequent occurence value of labels
toronto_merged['Cluster Labels']= toronto_merged['Cluster Labels'].astype(int)  # convert datatype of Labels from float to integer so that visualization can work properly

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


Visualize resulting clusters

In [109]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

In [122]:
toronto_merged['Cluster Labels'].value_counts() #count the distribution of labels


1    58
3    38
0     5
2     1
4     1
Name: Cluster Labels, dtype: int64

In [111]:
#cluster 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,0,Park,Coffee Shop,Chinese Restaurant,Indian Restaurant,Pharmacy,Fast Food Restaurant,Mobile Phone Shop,Music Store,Other Great Outdoors,Museum
20,North York,0,Park,Pool,Zoo,New American Restaurant,Organic Grocery,Optical Shop,Opera House,Office,Noodle House,Nightclub
31,North York,0,Park,Grocery Store,Shopping Mall,Gym / Fitness Center,Pizza Place,Vietnamese Restaurant,Bank,Coffee Shop,Noodle House,Other Great Outdoors
91,Etobicoke,0,Park,Eastern European Restaurant,Ice Cream Shop,Shopping Mall,Gym / Fitness Center,Bus Stop,Italian Restaurant,Noodle House,Other Great Outdoors,Organic Grocery
94,Etobicoke,0,Park,Hotel,Pizza Place,Clothing Store,Mexican Restaurant,Fish & Chips Shop,Bank,Theater,Gym,Grocery Store


In [112]:
#cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1,Italian Restaurant,Breakfast Spot,Park,Burger Joint,Playground,Newsagent,Organic Grocery,Optical Shop,Opera House,Office
9,Scarborough,1,Park,Ice Cream Shop,Restaurant,Café,Skating Rink,General Entertainment,Gym,Gym Pool,College Stadium,Thai Restaurant
16,Scarborough,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,North York,1,Coffee Shop,Clothing Store,Sandwich Place,Bakery,Japanese Restaurant,Fast Food Restaurant,Bank,Restaurant,Juice Bar,Liquor Store
21,North York,1,Korean Restaurant,Café,Park,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Fried Chicken Joint,Sandwich Place,Shopping Mall,Grocery Store
22,North York,1,Coffee Shop,Pizza Place,Korean Restaurant,Ramen Restaurant,Japanese Restaurant,Bubble Tea Shop,Fast Food Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Sandwich Place
26,North York,1,Coffee Shop,Japanese Restaurant,Burger Joint,Pizza Place,Rental Car Location,Fast Food Restaurant,Bank,Paper / Office Supplies Store,Gym,Greek Restaurant
27,North York,1,Restaurant,Gym,Coffee Shop,Sushi Restaurant,Supermarket,Intersection,Beer Store,Hockey Arena,History Museum,Park
29,North York,1,Coffee Shop,Furniture / Home Store,Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Falafel Restaurant,Bar,Massage Studio,Caribbean Restaurant,Sandwich Place
30,North York,1,Coffee Shop,Turkish Restaurant,Athletics & Sports,Liquor Store,Park,Gym,Sandwich Place,Chinese Restaurant,Bakery,Latin American Restaurant


In [113]:
#cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
102,Etobicoke Northwest,2,Hotel,Drugstore,Lounge,Coffee Shop,Pakistani Restaurant,Other Repair Shop,Other Great Outdoors,Organic Grocery,Optical Shop,Opera House


In [114]:
#cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3,Fast Food Restaurant,Coffee Shop,Trail,Bank,Hobby Shop,Gym,Chinese Restaurant,Sandwich Place,Paper / Office Supplies Store,Park
2,Scarborough,3,Pizza Place,Fast Food Restaurant,Bank,Fried Chicken Joint,Park,Food & Drink Shop,Sports Bar,Smoothie Shop,Supermarket,Beer Store
4,Scarborough,3,Bakery,Indian Restaurant,Gas Station,Coffee Shop,Bank,Grocery Store,Fried Chicken Joint,Music Store,Fast Food Restaurant,Pharmacy
5,Scarborough,3,Ice Cream Shop,Sandwich Place,Pizza Place,Grocery Store,Coffee Shop,Fast Food Restaurant,Restaurant,Japanese Restaurant,Bowling Alley,Intersection
6,Scarborough,3,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Pizza Place,Burger Joint,Hockey Arena,Grocery Store,Bank,Sandwich Place,Light Rail Station
7,Scarborough,3,Coffee Shop,Intersection,Bakery,Ice Cream Shop,Trail,Soccer Field,Mexican Restaurant,Metro Station,General Entertainment,Fast Food Restaurant
8,Scarborough,3,Pizza Place,Ice Cream Shop,Beach,Sports Bar,Park,Restaurant,Auto Garage,Newsagent,Organic Grocery,Optical Shop
10,Scarborough,3,Coffee Shop,Furniture / Home Store,Electronics Store,Pharmacy,Fast Food Restaurant,Light Rail Station,Asian Restaurant,Chinese Restaurant,Restaurant,Indian Restaurant
11,Scarborough,3,Pizza Place,Middle Eastern Restaurant,Burger Joint,Intersection,Grocery Store,Indian Chinese Restaurant,Restaurant,Sandwich Place,Coffee Shop,Bus Station
12,Scarborough,3,Chinese Restaurant,Shopping Mall,Bakery,Sandwich Place,Caribbean Restaurant,Indian Restaurant,Cantonese Restaurant,Mediterranean Restaurant,Sushi Restaurant,Supermarket


In [115]:
#cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,North York,4,Vietnamese Restaurant,Baseball Field,Restaurant,Zoo,New American Restaurant,Organic Grocery,Optical Shop,Opera House,Office,Noodle House
